In [6]:
# Código del archivo main.py para despliegue en AWS
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import pandas as pd
import joblib

app = FastAPI(title="Netflix Recommender API")

# 1. Cargar el Modelo y los Datos al iniciar el servidor
# Se asume que los archivos .pkl están en la misma carpeta
try:
    cosine_sim = joblib.load('cosine_sim_model.pkl')
    df_movies = pd.read_pickle('movie_data.pkl') # Contiene titulos y generos
    indices = pd.read_pickle('indices.pkl')      # Serie pandas indexada
    print("Modelo cargado exitosamente en memoria.")
except Exception as e:
    print(f"Error cargando modelo: {e}")

# 2. Definir el esquema de entrada
class MovieRequest(BaseModel):
    movie_title: str

# 3. Definir el Endpoint
@app.post("/recommend")
def recommend(request: MovieRequest):
    title = request.movie_title

    # Validación: ¿Existe la película?
    if title not in indices:
        raise HTTPException(status_code=404, detail="Película no encontrada en el catálogo")

    # Lógica de Recomendación
    idx = indices[title]
    if isinstance(idx, pd.Series):
        idx = idx.iloc[0]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Top 5 recomendaciones (excluyendo la propia película)
    top_indices = [i[0] for i in sim_scores[1:6]]

    recommendations = df_movies['title'].iloc[top_indices].tolist()

    return {
        "input_movie": title,
        "recommendations": recommendations
    }

Modelo cargado exitosamente en memoria.


In [ ]:
# 3. Iniciar Uvicorn en un proceso separado
# Usamos nohup y & para que corra en background y no bloquee la celda
# Guardamos el PID para poder terminarlo después

# Aseguramos que no haya procesos anteriores corriendo en el puerto 8000
# Agregamos 'if' para evitar error si no hay proceso y mostrar un mensaje claro
pid_on_port = !lsof -t -i:8000
if pid_on_port:
    try:
        !kill $(lsof -t -i:8000) 2>/dev/null
        print(f"Proceso anterior en el puerto 8000 ({pid_on_port[0]}) terminado.")
    except Exception as e:
        print(f"Advertencia: No se pudo terminar el proceso anterior en el puerto 8000: {e}")
else:
    print("No se encontraron procesos anteriores en el puerto 8000.")


uvicorn_log_file_stdout = open("uvicorn_stdout.log", "w")
uvicorn_log_file_stderr = open("uvicorn_stderr.log", "w")

process = subprocess.Popen(
    ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"],
    stdout=uvicorn_log_file_stdout,
    stderr=uvicorn_log_file_stderr
)
uvicorn_pid = process.pid
print(f"Uvicorn iniciado con PID: {uvicorn_pid}")

# Esperar de forma más robusta a que el servidor se inicie completamente
url_check = "http://127.0.0.1:8000/" # Un endpoint base para verificar si responde
max_retries = 15 # Aumentar el número de reintentos
retry_interval = 1 # Esperar 1 segundo entre reintentos

server_ready = False
for i in range(max_retries):
    try:
        # Intenta conectar al servidor. Un 200 indica que Uvicorn está sirviendo.
        response = requests.get(url_check, timeout=1) # Timeout corto para cada intento
        if response.status_code == 200:
            server_ready = True
            print(f"Servidor Uvicorn está listo y respondiendo (intento {i+1}/{max_retries}).")
            break
    except requests.exceptions.ConnectionError:
        pass # Ignorar errores de conexión si el servidor no está listo
    except requests.exceptions.ReadTimeout:
        print(f"Servidor Uvicorn en el puerto 8000 está iniciando, pero tardó en responder. (intento {i+1}/{max_retries})")
    except Exception as e:
        print(f"Error inesperado al verificar el servidor: {e}")

    time.sleep(retry_interval)

uvicorn_log_file_stdout.close()
uvicorn_log_file_stderr.close()

if not server_ready:
    print("\n¡Error! El servidor Uvicorn no se pudo iniciar o no respondió a tiempo.")
    print("Por favor, revise los logs de error de Uvicorn para más detalles:")
    try:
        with open("uvicorn_stderr.log", "r") as f:
            error_logs = f.read()
            if error_logs:
                print("---- uvicorn_stderr.log ----")
                print(error_logs)
                print("----------------------------")
            else:
                print("uvicorn_stderr.log está vacío, lo que podría indicar un problema diferente o que no se generó ningún error explícito.")
    except FileNotFoundError:
        print("El archivo uvicorn_stderr.log no fue encontrado, lo que es inesperado.")

    raise Exception("Fallo crítico: El servidor Uvicorn no está listo para recibir solicitudes.")
else:
    print("Servidor Uvicorn debería estar en ejecución y listo para solicitudes.")

In [1]:
# 4. Enviar una solicitud POST al endpoint /recommend

# Asegúrate de usar un título de película que exista en tu 'indices'
# Por ejemplo, 'Minions' o 'Wonder Woman' según tu df_movies

# Puedes ver algunos títulos válidos ejecutando:
# display(indices.head())

movie_to_recommend = "Minions" # Puedes cambiar este título

url = "http://127.0.0.1:8000/recommend"
headers = {"Content-Type": "application/json"}
payload = {"movie_title": movie_to_recommend}

try:
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()  # Lanza una excepción para errores HTTP
    print("Respuesta del servidor:")
    print(response.json())
except requests.exceptions.RequestException as e:
    print(f"Error al conectar o recibir respuesta del servidor: {e}")
    if hasattr(e, 'response') and e.response is not None:
        print(f"Contenido del error: {e.response.text}")

NameError: name 'requests' is not defined

In [2]:
# 5. Detener el proceso de Uvicorn
# Este comando usa la variable 'uvicorn_pid' de la celda anterior.
# Asegúrate de ejecutar la celda 'Iniciar el servidor Uvicorn' antes.

if 'uvicorn_pid' in globals() and uvicorn_pid is not None:
    try:
        os.kill(uvicorn_pid, 9) # Envía la señal SIGKILL (9) para terminar el proceso
        print(f"Proceso Uvicorn (PID: {uvicorn_pid}) terminado.")
        uvicorn_pid = None # Limpiar la variable para evitar reintentos
    except OSError as e:
        print(f"Error al intentar terminar el proceso {uvicorn_pid}: {e}")
else:
    print("No se encontró un PID de Uvicorn para terminar.")


No se encontró un PID de Uvicorn para terminar.


In [3]:
import subprocess
import time
import requests
import os

# Aseguramos que no haya procesos anteriores corriendo en el puerto 8000
# Agregamos 'if' para evitar error si no hay proceso y mostrar un mensaje claro
pid_on_port = !lsof -t -i:8000
if pid_on_port:
    try:
        !kill $(lsof -t -i:8000) 2>/dev/null
        print(f"Proceso anterior en el puerto 8000 ({pid_on_port[0]}) terminado.")
    except Exception as e:
        print(f"Advertencia: No se pudo terminar el proceso anterior en el puerto 8000: {e}")
else:
    print("No se encontraron procesos anteriores en el puerto 8000.")

# 3. Iniciar Uvicorn en un proceso separado
# Redirigimos la salida a archivos para depuración
uvicorn_log_file_stdout = open("uvicorn_stdout.log", "w")
uvicorn_log_file_stderr = open("uvicorn_stderr.log", "w")

process = subprocess.Popen(
    ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"],
    stdout=uvicorn_log_file_stdout,
    stderr=uvicorn_log_file_stderr
)
uvicorn_pid = process.pid
print(f"Uvicorn iniciado con PID: {uvicorn_pid}")

# Esperar de forma más robusta a que el servidor se inicie completamente
url_check = "http://127.0.0.1:8000/" # Un endpoint base para verificar si responde
max_retries = 15 # Aumentar el número de reintentos
retry_interval = 1 # Esperar 1 segundo entre reintentos

server_ready = False
for i in range(max_retries):
    try:
        # Intenta conectar al servidor. Un 200 indica que Uvicorn está sirviendo.
        response = requests.get(url_check, timeout=1) # Timeout corto para cada intento
        if response.status_code == 200:
            server_ready = True
            print(f"Servidor Uvicorn está listo y respondiendo (intento {i+1}/{max_retries}).")
            break
    except requests.exceptions.ConnectionError:
        pass # Ignorar errores de conexión si el servidor no está listo
    except requests.exceptions.ReadTimeout:
        print(f"Servidor Uvicorn en el puerto 8000 está iniciando, pero tardó en responder. (intento {i+1}/{max_retries})")
    except Exception as e:
        print(f"Error inesperado al verificar el servidor: {e}")

    time.sleep(retry_interval)

# Cerrar los archivos de log para asegurar que se escriban completamente
uvicorn_log_file_stdout.close()
uvicorn_log_file_stderr.close()

if not server_ready:
    print("\n¡Error! El servidor Uvicorn no se pudo iniciar o no respondió a tiempo.")
    print("Por favor, revise los logs de error de Uvicorn para más detalles:")
    try:
        with open("uvicorn_stderr.log", "r") as f:
            error_logs = f.read()
            if error_logs:
                print("---- uvicorn_stderr.log ----")
                print(error_logs)
                print("----------------------------")
            else:
                print("uvicorn_stderr.log está vacío, lo que podría indicar un problema diferente o que no se generó ningún error explícito.")
    except FileNotFoundError:
        print("El archivo uvicorn_stderr.log no fue encontrado, lo que es inesperado.")

    # Si no se pudo iniciar el servidor, es un fallo crítico, detendremos la ejecución.
    # Para depuración, podríamos no detener y dejar que el usuario revise los logs.
    # Por ahora, levantamos una excepción para señalar el problema.
    raise Exception("Fallo crítico: El servidor Uvicorn no está listo para recibir solicitudes.")
else:
    print("Servidor Uvicorn debería estar en ejecución y listo para solicitudes.")

No se encontraron procesos anteriores en el puerto 8000.
Uvicorn iniciado con PID: 5168

¡Error! El servidor Uvicorn no se pudo iniciar o no respondió a tiempo.
Por favor, revise los logs de error de Uvicorn para más detalles:
---- uvicorn_stderr.log ----
INFO:     Started server process [5168]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)

----------------------------


Exception: Fallo crítico: El servidor Uvicorn no está listo para recibir solicitudes.

In [4]:
# 4. Enviar una solicitud POST al endpoint /recommend

# Asegúrate de usar un título de película que exista en tu 'indices'
# Por ejemplo, 'Minions' o 'Wonder Woman' según tu df_movies

# Puedes ver algunos títulos válidos ejecutando:
# display(indices.head())

movie_to_recommend = "Minions" # Puedes cambiar este título

url = "http://127.0.0.1:8000/recommend"
headers = {"Content-Type": "application/json"}
payload = {"movie_title": movie_to_recommend}

try:
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()  # Lanza una excepción para errores HTTP
    print("Respuesta del servidor:")
    print(response.json())
except requests.exceptions.RequestException as e:
    print(f"Error al conectar o recibir respuesta del servidor: {e}")
    if hasattr(e, 'response') and e.response is not None:
        print(f"Contenido del error: {e.response.text}")

Respuesta del servidor:
{'input_movie': 'Minions', 'recommendations': ['Despicable Me 3', 'Banana', 'Minions: Orientation Day', 'Despicable Me 2', 'The Merry Gentleman']}


In [5]:
# 5. Detener el proceso de Uvicorn
# Este comando usa la variable 'uvicorn_pid' de la celda anterior.
# Asegúrate de ejecutar la celda 'Iniciar el servidor Uvicorn' antes.

if 'uvicorn_pid' in globals() and uvicorn_pid is not None:
    try:
        os.kill(uvicorn_pid, 9) # Envía la señal SIGKILL (9) para terminar el proceso
        print(f"Proceso Uvicorn (PID: {uvicorn_pid}) terminado.")
        uvicorn_pid = None # Limpiar la variable para evitar reintentos
    except OSError as e:
        print(f"Error al intentar terminar el proceso {uvicorn_pid}: {e}")
else:
    print("No se encontró un PID de Uvicorn para terminar.")


Proceso Uvicorn (PID: 5168) terminado.


In [3]:
import subprocess
import time
import requests
import os

# 1. Instalar las librerías necesarias
!pip install fastapi uvicorn requests

### Guardar el código de la aplicación en `main.py`

In [4]:
# 2. Guardar el contenido del código 'main.py' en un archivo
main_py_content = """
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import pandas as pd
import joblib

app = FastAPI(title="Netflix Recommender API")

# 1. Cargar el Modelo y los Datos al iniciar el servidor
# Se asume que los archivos .pkl están en la misma carpeta
try:
    cosine_sim = joblib.load('cosine_sim_model.pkl')
    df_movies = pd.read_pickle('movie_data.pkl') # Contiene titulos y generos
    indices = pd.read_pickle('indices.pkl')      # Serie pandas indexada
    print("Modelo cargado exitosamente en memoria.")
except Exception as e:
    print(f"Error cargando modelo: {e}")

# 2. Definir el esquema de entrada
class MovieRequest(BaseModel):
    movie_title: str

# 3. Definir el Endpoint
@app.post("/recommend")
def recommend(request: MovieRequest):
    title = request.movie_title

    # Validación: ¿Existe la película?
    if title not in indices:
        raise HTTPException(status_code=404, detail="Película no encontrada en el catálogo")

    # Lógica de Recomendación
    idx = indices[title]
    if isinstance(idx, pd.Series):
        idx = idx.iloc[0]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Top 5 recomendaciones (excluyendo la propia película)
    top_indices = [i[0] for i in sim_scores[1:6]]

    recommendations = df_movies['title'].iloc[top_indices].tolist()

    return {
        "input_movie": title,
        "recommendations": recommendations
    }
"""

with open("main.py", "w") as f:
    f.write(main_py_content)

print("Archivo 'main.py' creado exitosamente.")

Archivo 'main.py' creado exitosamente.


### Iniciar el servidor Uvicorn en segundo plano

In [ ]:
import subprocess
import time
import requests
import os

# Aseguramos que no haya procesos anteriores corriendo en el puerto 8000
# Agregamos 'if' para evitar error si no hay proceso y mostrar un mensaje claro
pid_on_port = !lsof -t -i:8000
if pid_on_port:
    try:
        !kill $(lsof -t -i:8000) 2>/dev/null
        print(f"Proceso anterior en el puerto 8000 ({pid_on_port[0]}) terminado.")
    except Exception as e:
        print(f"Advertencia: No se pudo terminar el proceso anterior en el puerto 8000: {e}")
else:
    print("No se encontraron procesos anteriores en el puerto 8000.")

# 3. Iniciar Uvicorn en un proceso separado
# Redirigimos la salida a archivos para depuración
uvicorn_log_file_stdout = open("uvicorn_stdout.log", "w")
uvicorn_log_file_stderr = open("uvicorn_stderr.log", "w")

process = subprocess.Popen(
    ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"],
    stdout=uvicorn_log_file_stdout,
    stderr=uvicorn_log_file_stderr
)
uvicorn_pid = process.pid
print(f"Uvicorn iniciado con PID: {uvicorn_pid}")

# Esperar de forma más robusta a que el servidor se inicie completamente
url_check = "http://127.0.0.1:8000/" # Un endpoint base para verificar si responde
max_retries = 15 # Aumentar el número de reintentos
retry_interval = 1 # Esperar 1 segundo entre reintentos

server_ready = False
for i in range(max_retries):
    try:
        # Intenta conectar al servidor. Un 200 indica que Uvicorn está sirviendo.
        response = requests.get(url_check, timeout=1) # Timeout corto para cada intento
        if response.status_code == 200:
            server_ready = True
            print(f"Servidor Uvicorn está listo y respondiendo (intento {i+1}/{max_retries}).")
            break
    except requests.exceptions.ConnectionError:
        pass # Ignorar errores de conexión si el servidor no está listo
    except requests.exceptions.ReadTimeout:
        print(f"Servidor Uvicorn en el puerto 8000 está iniciando, pero tardó en responder. (intento {i+1}/{max_retries})")
    except Exception as e:
        print(f"Error inesperado al verificar el servidor: {e}")

    time.sleep(retry_interval)

# Cerrar los archivos de log para asegurar que se escriban completamente
uvicorn_log_file_stdout.close()
uvicorn_log_file_stderr.close()

if not server_ready:
    print("\n¡Error! El servidor Uvicorn no se pudo iniciar o no respondió a tiempo.")
    print("Por favor, revise los logs de error de Uvicorn para más detalles:")
    try:
        with open("uvicorn_stderr.log", "r") as f:
            error_logs = f.read()
            if error_logs:
                print("---- uvicorn_stderr.log ----")
                print(error_logs)
                print("----------------------------")
            else:
                print("uvicorn_stderr.log está vacío, lo que podría indicar un problema diferente o que no se generó ningún error explícito.")
    except FileNotFoundError:
        print("El archivo uvicorn_stderr.log no fue encontrado, lo que es inesperado.")

    # Si no se pudo iniciar el servidor, es un fallo crítico, detendremos la ejecución.
    # Para depuración, podríamos no detener y dejar que el usuario revise los logs.
    # Por ahora, levantamos una excepción para señalar el problema.
    raise Exception("Fallo crítico: El servidor Uvicorn no está listo para recibir solicitudes.")
else:
    print("Servidor Uvicorn debería estar en ejecución y listo para solicitudes.")


### Enviar una solicitud de prueba

In [6]:
# 4. Enviar una solicitud POST al endpoint /recommend

# Asegúrate de usar un título de película que exista en tu 'indices'
# Por ejemplo, 'Minions' o 'Wonder Woman' según tu df_movies

# Puedes ver algunos títulos válidos ejecutando:
# display(indices.head())

movie_to_recommend = "The Dark Knight" # Puedes cambiar este título

url = "http://127.0.0.1:8000/recommend"
headers = {"Content-Type": "application/json"}
payload = {"movie_title": movie_to_recommend}

try:
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()  # Lanza una excepción para errores HTTP
    print("Respuesta del servidor:")
    print(response.json())
except requests.exceptions.RequestException as e:
    print(f"Error al conectar o recibir respuesta del servidor: {e}")
    if hasattr(e, 'response') and e.response is not None:
        print(f"Contenido del error: {e.response.text}")

Error al conectar o recibir respuesta del servidor: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /recommend (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x788530836b10>: Failed to establish a new connection: [Errno 111] Connection refused'))


### Detener el servidor Uvicorn

In [7]:
# 5. Detener el proceso de Uvicorn
# Este comando usa la variable 'uvicorn_pid' de la celda anterior.
# Asegúrate de ejecutar la celda 'Iniciar el servidor Uvicorn' antes.

if 'uvicorn_pid' in globals() and uvicorn_pid is not None:
    try:
        os.kill(uvicorn_pid, 9) # Envía la señal SIGKILL (9) para terminar el proceso
        print(f"Proceso Uvicorn (PID: {uvicorn_pid}) terminado.")
        uvicorn_pid = None # Limpiar la variable para evitar reintentos
    except OSError as e:
        print(f"Error al intentar terminar el proceso {uvicorn_pid}: {e}")
else:
    print("No se encontró un PID de Uvicorn para terminar.")


Proceso Uvicorn (PID: 4652) terminado.
